# Part 0: Bulk Loading BibTeX data

Load data from a .bib file into an sqlite database.

Based on the [documentation of the BibTexParser Python package](https://bibtexparser.readthedocs.io/en/master/tutorial.html#step-2-parse-it).

In [1]:
__author__ = "Christine Mendoza, \
    with some sqlite3-related code from Dr. Gary Bishop's / Dr. John Majikes' UNC Chapel Hill COMP421 (Databases) class"

Before going further, make sure to run the following commands in the terminal for package installation and restart if necessary:

```pip install bibtexparser```

```pip install sqlite3```

Also make sure to edit your file paths as necessary below.

In [2]:
RAW_BIB_DATA: str = "./data/paper-selection-analysis/part-0.bib"
OUTPUT_DB: str = "./data/sqlite/analysis.db"

## Step 0: Parse BibTeX data

NOTE: The BiBTeX file should not have any spaces in its fields (possible with Scopus for fields related to funding), and duplicate fields should not be present in entries. Please ensure the file is formed properly before parsing, or entries will be omitted. I chose to omit the fields on funding_details and funding_text entirely.

In [3]:
import bibtexparser

with open(RAW_BIB_DATA) as bibtex_file:
    bib_to_dict = bibtexparser.load(bibtex_file)

# Check that the number of entries loaded in is as expected.
print(len(bib_to_dict.entries))

308


## Step 1: Determine columns for part 0 sqlite database

Not all BibTeX entries have the same columns. If you do not know the columns you would like in the database in advance, you will have to determine this from the columns present in the BibTeX data.

In [4]:
raw_columns: set = set([])

for entry in bib_to_dict.entries:
    current_keys: list[str] = entry.keys()

    for key in current_keys:
        raw_columns.add(key)

# print result
print(raw_columns)

{'journal', 'chemicals_cas', 'issn', 'number', 'volume', 'language', 'references', 'pubmed_id', 'document_type', 'sponsors', 'publisher', 'coden', 'author_keywords', 'abbrev_source_title', 'year', 'page_count', 'affiliation', 'ENTRYTYPE', 'keywords', 'correspondence_address1', 'author', 'title', 'isbn', 'pages', 'ID', 'source', 'note', 'abstract', 'art_number', 'editor', 'doi', 'url'}


Decide on your columns, then edit `final_columns` below as necessary, keeping in mind reserved keywords in sql (ex. `references`). 

`final_columns` is currently assigned the value of `raw_columns`, with an edit to rename `references` to `paper_references`, if it exists in the `raw_columns` set.

In [5]:
final_columns: set = raw_columns

if "references" in final_columns:
    final_columns.remove("references")
    final_columns.add("paper_references")

# print result
print(final_columns)

{'journal', 'chemicals_cas', 'issn', 'number', 'volume', 'language', 'pubmed_id', 'document_type', 'sponsors', 'publisher', 'coden', 'author_keywords', 'abbrev_source_title', 'year', 'page_count', 'affiliation', 'ENTRYTYPE', 'keywords', 'correspondence_address1', 'author', 'title', 'isbn', 'paper_references', 'pages', 'ID', 'source', 'note', 'abstract', 'art_number', 'editor', 'doi', 'url'}


## Step 2: Create sqlite database and set up part 0 table

Set up your database connection. A file with the path specified by `OUTPUT_DB` will automatically be created if one does not exist.

In [6]:
import sqlite3
db = sqlite3.Connection(OUTPUT_DB)
cursor = db.cursor()

To speed up the process, I assume here that we're treating all columns in `final_columns` as text and join together the set elements to create a string. It's usually best practice to simply write out the schema yourself and to avoid string concatenation because of the risk of SQL injection. But since this is a local file, I am assuming no one else is tampering with this file.

In [7]:
bibtex_fields = " TEXT, ".join(final_columns)

create_tables = """CREATE TABLE All_Papers(paper_analysis_id INTEGER PRIMARY KEY AUTOINCREMENT, """ + bibtex_fields + """ TEXT);"""

# print resulting (unformatted) query
print(create_tables)

CREATE TABLE All_Papers(paper_analysis_id INTEGER PRIMARY KEY AUTOINCREMENT, journal TEXT, chemicals_cas TEXT, issn TEXT, number TEXT, volume TEXT, language TEXT, pubmed_id TEXT, document_type TEXT, sponsors TEXT, publisher TEXT, coden TEXT, author_keywords TEXT, abbrev_source_title TEXT, year TEXT, page_count TEXT, affiliation TEXT, ENTRYTYPE TEXT, keywords TEXT, correspondence_address1 TEXT, author TEXT, title TEXT, isbn TEXT, paper_references TEXT, pages TEXT, ID TEXT, source TEXT, note TEXT, abstract TEXT, art_number TEXT, editor TEXT, doi TEXT, url TEXT);


Finally, we can create the table!

In [8]:
cursor.executescript(create_tables)

Check that the table was actually created.

In [9]:
cursor.execute('''
               SELECT COUNT(*)
               FROM sqlite_master
               WHERE type='table' AND
                     name NOT LIKE 'sqlite_%' ''')
table_count = cursor.fetchone()[0]
print(f'There is/are {table_count} table(s) in the database.')

There is/are 1 table(s) in the database.


## Step 3: Actually bulk load the data

We format each entry to match the table schema and insert it as a record into the part_0 table in the database.

In [10]:
# We assume mutually exclusive access to the database.

for current_entry in bib_to_dict.entries:
    entry = current_entry.copy()

    # format the entry to match the table schema
    for key in final_columns:
        if key not in entry:
            entry[key] = ""
    
    
    # I just wrote it all out this time, with a little help from regex
    cursor.execute('''INSERT INTO All_Papers (correspondence_address1, number, doi, isbn, 
                        year, abbrev_source_title, editor, art_number, ID, pubmed_id, volume, 
                        coden, publisher, author, page_count, paper_references, issn, source, 
                        language, author_keywords, keywords, url, note, 
                        affiliation, pages, ENTRYTYPE, abstract, sponsors, document_type, 
                        title, journal) 
                    values (?, ?, ?, ?, ?,
                            ?, ?, ?, ?, ?,
                            ?, ?, ?, ?, ?,
                            ?, ?, ?, ?, ?,
                            ?, ?, ?, ?, ?,
                            ?, ?, ?, ?, ?,
                            ?)''',
                    (entry['correspondence_address1'],
                        entry['number'],
                        entry['doi'],
                        entry['isbn'],
                        entry['year'],
                        entry['abbrev_source_title'],
                        entry['editor'],
                        entry['art_number'],
                        entry['ID'],
                        entry['pubmed_id'],
                        entry['volume'],
                        entry['coden'],
                        entry['publisher'],
                        entry['author'],
                        entry['page_count'],
                        entry['paper_references'],
                        entry['issn'],
                        entry['source'],
                        entry['language'],
                        entry['author_keywords'],
                        entry['keywords'],
                        entry['url'],
                        entry['note'],
                        entry['affiliation'],
                        entry['pages'],
                        entry['ENTRYTYPE'],
                        entry['abstract'],
                        entry['sponsors'],
                        entry['document_type'],
                        entry['title'],
                        entry['journal']))

Verify your table. You should have the same amount of records in your table as you did entries in the original BibTeX file.

In [11]:
table_size: int = cursor.execute('''select count(*) from %s''' % "All_Papers").fetchone()[0]
print(table_size)

assert(table_size == len(bib_to_dict.entries))

308


## Conclusion

Now that you have all the records loaded into a table in your database, you can proceed to the next part!